In [1]:
library(MASS) # Load MASS package
data<-biopsy # initialize biopsy data


Warning message:
"package 'MASS' was built under R version 3.6.3"

In [2]:
# Scale function using min-max transform between 0 and 1
scale.0.1<-function(x){
  (x-min(x,na.rm=TRUE))/(max(x,na.rm=TRUE)-min(x,na.rm=TRUE))
}


In [3]:
# Normalize data
norm.data<-data.frame(lapply(data[,2:10], function(x) scale.0.1(x)))
norm.data$class<-data$class
str(norm.data) # Checking the structure
sum(is.na(norm.data)) # Checking the sum
norm.data <- norm.data[complete.cases(norm.data),] 


'data.frame':	699 obs. of  10 variables:
 $ V1   : num  0.444 0.444 0.222 0.556 0.333 ...
 $ V2   : num  0 0.333 0 0.778 0 ...
 $ V3   : num  0 0.333 0 0.778 0 ...
 $ V4   : num  0 0.444 0 0 0.222 ...
 $ V5   : num  0.111 0.667 0.111 0.222 0.111 ...
 $ V6   : num  0 1 0.111 0.333 0 ...
 $ V7   : num  0.222 0.222 0.222 0.222 0.222 ...
 $ V8   : num  0 0.111 0 0.667 0 ...
 $ V9   : num  0 0 0 0 0 ...
 $ class: Factor w/ 2 levels "benign","malignant": 1 1 1 1 1 2 1 1 1 1 ...


[1] 16

In [11]:
library(nnet)

Warning message:
"package 'nnet' was built under R version 3.6.3"

In [4]:
# Split the dataset to Train and Test
set.seed(42)
index<-sample(1:nrow(norm.data),round(0.75*nrow(norm.data)),replace=FALSE)
train<-norm.data[index,]
test<-norm.data[-index,]


In [12]:
# Function that take a training and test data set and a range of integer corresponding to the number of neurons to be used as parameter.
cross.val.nnet<-function(train,test,low_range,high_range){
acc<-NULL
for (h in low_range:high_range) {
temp.nn<-nnet(class~.,size=h,data=train) #train a neural network made with nnet()
pred<-predict(temp.nn,test,type="class") # make prediction on test set
Table<-table(test$class,pred)
accuracy<-sum(diag(Table))/sum(Table) #return the accuracy of the prediction.
acc<-c(acc,accuracy)
  }
return(acc)
}


In [13]:
cross.val.nnet

function(train,test,low_range,high_range){
acc<-NULL
for (h in low_range:high_range) {
temp.nn<-nnet(class~.,size=h,data=train) #train a neural network made with nnet()
pred<-predict(temp.nn,test,type="class") # make prediction on test set
Table<-table(test$class,pred)
accuracy<-sum(diag(Table))/sum(Table) #return the accuracy of the prediction.
acc<-c(acc,accuracy)
  }
return(acc)
}

In [14]:
set.seed(42)
cross.val.nnet(train,test,1,9) # From the results, we could use 2 or 6 neurons in the hidden layer


# weights:  12
initial  value 364.256592 
iter  10 value 37.801784
iter  20 value 35.137607
iter  30 value 34.866657
iter  40 value 34.645363
iter  50 value 34.346505
iter  60 value 34.321919
iter  70 value 34.313870
iter  80 value 34.257308
iter  90 value 34.256719
iter 100 value 34.220338
final  value 34.220338 
stopped after 100 iterations
# weights:  23
initial  value 335.960256 
iter  10 value 55.598548
iter  20 value 47.596276
iter  30 value 47.516059
iter  40 value 47.461438
iter  50 value 45.744461
iter  60 value 44.832821
iter  70 value 44.825953
iter  80 value 44.824357
final  value 44.823859 
converged
# weights:  34
initial  value 336.224638 
iter  10 value 30.674457
iter  20 value 26.286168
iter  30 value 23.763237
iter  40 value 22.796802
iter  50 value 22.088646
iter  60 value 21.801078
iter  70 value 21.631699
iter  80 value 21.629902
iter  90 value 21.629873
iter 100 value 21.629831
final  value 21.629831 
stopped after 100 iterations
# weights:  45
initial  value 343.

[1] 0.9649123 0.9590643 0.9590643 0.9532164 0.9649123 0.9649123 0.9356725
[8] 0.9590643 0.9590643

In [15]:
library(neuralnet)  
train$class <- as.numeric(train$class)
test$class <- as.numeric(test$class)
n <- names(train)
# This function can only handle numeric value. Can’t deal with factors.
f <- as.formula(paste("class ~", paste(n[!n %in% "class"], collapse = " + "))) 
set.seed(42)
model.neuralnet.1<-neuralnet(f,data=train,hidden=6) # Used 6 neurons


Warning message:
"package 'neuralnet' was built under R version 3.6.3"

In [16]:
# compute() function to make prediction 
pred.model.1 <- compute(model.neuralnet.1,test[,1:9]) 
pred.model.1$net.result <-round(pred.model.1$net.result)
Table<-table(test$class,pred.model.1$net.result)
Table


   
      1   2   3
  1 101  10   0
  2  12  47   1

In [17]:
accuracy<-sum(diag(Table))/sum(Table) # compute the accuracy of your model
accuracy


[1] 0.8654971

In [18]:
set.seed(42)
model.neuralnet.2<-neuralnet(f, data=train, hidden=6, algorithm ='rprop-')
pred.model.2 <- compute(model.neuralnet.2,test[,1:9])
pred.model.2$net.result <-round(pred.model.2$net.result)
pred.model.2$net.result[which(pred.model.2$net.result<=1)]<- 1
pred.model.2$net.result[which(pred.model.2$net.result>=2)]<- 2
Table<-table(test$class,pred.model.2$net.result)
Table


   
      1   2
  1 105   6
  2   4  56

In [19]:
accuracy<-sum(diag(Table))/sum(Table)
accuracy


[1] 0.9415205

In [20]:
set.seed(42)
model.neuralnet.5<-neuralnet(f, data=train, hidden=6, algorithm ='rprop-', learningrate=0.001)
pred.model.5 <- compute(model.neuralnet.5,test[,1:9])
pred.model.5$net.result <-round(pred.model.5$net.result)
Table<-table(test$class,pred.model.5$net.result)
Table


   
      0   1   2   3
  1   1 104   5   1
  2   0   4  54   2

In [21]:
pred.model.5$net.result[which(pred.model.5$net.result<=1)]<-1
pred.model.5$net.result[which(pred.model.5$net.result>=2)]<-2
Table<-table(test$class,pred.model.5$net.result)
Table


   
      1   2
  1 105   6
  2   4  56

In [22]:
accuracy<-sum(diag(Table))/sum(Table)
accuracy


[1] 0.9415205

In [23]:
set.seed(42)
model.neuralnet.6<-neuralnet(f, data=train, hidden=6, algorithm ='rprop-', learningrate=1)
pred.model.6 <- compute(model.neuralnet.6,test[,1:9])
pred.model.6$net.result <-round(pred.model.6$net.result)
Table<-table(test$class,pred.model.6$net.result)
Table


   
      0   1   2   3
  1   1 104   5   1
  2   0   4  54   2

In [24]:
pred.model.6$net.result[which(pred.model.6$net.result==0)]<-1
Table<-table(test$class,pred.model.6$net.result)
Table


   
      1   2   3
  1 105   5   1
  2   4  54   2

In [25]:
accuracy<-sum(diag(Table))/sum(Table)
accuracy
plot(model.neuralnet.5) # Visual representation of the neural network


[1] 0.9298246

In [26]:
plot(model.neuralnet.5)

In [27]:
# create FF NN with three hidden layers of nine neurons
model.neuralnet.7<-neuralnet(f, data=train, hidden=c(9,9,9)) 
pred.model.7 <- compute(model.neuralnet.7,test[,1:9])
pred.model.7$net.result <-round(pred.model.7$net.result) 
Table<-table(test$class,pred.model.7$net.result) 
Table


   
      1   2
  1 107   4
  2   4  56

In [28]:
plot(model.neuralnet.7)# Plot the FF NN of the above